# Deep Residual Learning for Image Recognition

# Abstract(논문 개요)
- 심층 신경망은 훈련하기 어렵다.
- 우리는 이전에 사용된 것보다 훨씬 더 깊은 네트워크의 훈련을 쉽게하기 위해서 residual learning 프레임워크를 제시한다.
- 참조되지 않은 함수를 학습하는 것 대신에, layer의 입력값을 참조하여 residual 함수를 학습하는 것으로써 layer를 명확하게 새로 만든다.
- 우리는 종합적인 경험적 증거를 제시한다. 이 residual network는 최적화하는 데 쉽고 꽤 증가된 깊이를 통해 정확도를 얻어내는 것이 가능하다는.
- ImageNet 데이터셋에서, 우리는 VGG 네트워크보다 8배 더 깊은 최대 152개의 층으로 residual 네트워크를 평가하지만 여전히 복잡성은 낮다.
- 이러한 residual 네트워크의 앙상블은 ImageNet 테스트 데이터셋에 대하여 3.57%의 오류를 달성했다.
- 이 결과는 ILSVRC 2015 분류 과제에서 1위를 차지했다.
- 또한 CIFAR-10 데이터에 대한 분석을 100개와 1000개의 layer로 제공한다.
- 표현(representation)의 깊이는 많은 시각적 인식 과제에서 매우 중요하다.
- 단지 극도로 깊은 표현 덕분에, 우리는 COCO 객체 탐지 데이터셋에서 상대적으로 28% 향상하게 되었다.
- Deep residual 네트워크는 ILSVRC & COCO 2015 대회에 제출한 기초이다. 또한 우리는 ImageNet 탐지, ImageNet localization, COCO 탐지, COCO segmentation의 과제에서 1등을 한 이유이다.

> 요점 정리 : residual learning framework는 VGG 네트워크보다 깊지만 복잡성은 낮다. 성능은 더 좋다. residual 네트워크는 최적화가 쉽고 깊어진 깊이를 통해 정확도가 높아졌다.

# 1. Introduction
- deep convolutional neural network는 이미지 분류(image classification)의 일련의 돌파구라고 여겨졌다.
- deep network는 자연스럽게 저/중/고 수준의 feature과 종단간 다층 방식(end-to-end multilayer fashion)으로 통합시켰다. 특징의 "수준"은 쌓인 레이어의 깊이의 수에 의해 강화시킬 수 있다.
- 최근의 증거들에 따르면 네트워크의 깊이가 매우 중요하고 까다로운 ImageNet 데이터셋에 대한 주요 결과는 모두 "매우 깊은" 모델을 활용한다. 깊이는 16 ~ 30이다.
- 다른 사소하지 않은 시각적 인식 작업도 심층 모델(very deep model)의 장점을 많이 받았다.
- 깊이의 중요성에 대한 한 가지 질문이 있다. 더 많은 layer를 쌓는 것만큼 더 나은 네트워크를 배우는 것이 쉬운가?
- 이 질문의 장애물은 처음부터 수렴을 방해하는 vanishing/exploding gradient 라는 악명 높은 문제이다.
- 그러나, 이 문제는 정규화된 초기화(normalized initialization)과 중간 정규화 계층(intermediate normalization layer)에 의해 주로 해결되었다. 이는 수십 개의 계층을 가진 네트워크가 역전파(backpropagation)로 확률적 경사하강법(SGD, stochastic gradient descent)를 위해 수렴을 시작할 수 있게 해준다.
- 더 깊은 네트워크가 수렴을 시작할 수 있을 때, 성능저하(degradation) 문제가 드러났다. 네트워크의 깊이가 증가하면, 정확도가 포화되고(놀랍지 않을 수 있음) 빠르게 저하된다.
- 예상외로 이러한 degradation 문제는 과적합(overfitting)에 의해 발생한 것이 아니고, 적절하게 deep model에 더 많은 layer를 추가하는 것이 훈련 오류를 증가시킨다는 것이 우리의 실험을 통해 철저히 검증되었다.
- 그림 1은 전형적인 예시를 보여준다.
- <img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FF41Yc%2FbtqDCZoURC2%2FkjRdYZGu7c0IXCbvK11x71%2Fimg.png" width=400>

  - 그림 1. 
  - 왼쪽 : 20층과 56층의 기본(plain) 네트워크를 사용하는 CIFAR-10 데이터셋의 훈련 오류
  - 오른쪽 : 테스트 오류
  - 심층 네트워크는 훈련 오류가 높으므로, 테스트 오류가 발생한다.
  - ImageNet에서 유사한 현상이 그림.4.에 나와있다.
- 훈련 저하도의 저하는 모든 시스템이 비슷하게 최적화하기 쉬운 것은 아니라는 것을 나타낸다.
- 더 얕은 구조(architecture)와 더 많은 layer를 추가하는 더 깊은 대응물(counterpart)를 고려해보자.
- 더 깊은 모델에 대한 구성별 해결방법이 있다: 추가된 layer는 identity mapping(덧셈 연산), 다른 layer는 학습된 얕은 모델에서 복사
- 이 구성된 해결방법의 존재는 더 깊은 모델이 더 낮은 대응품(counterpart)보다 더 높은 훈련 오류를 만들어내면 안된다는 것을 나타낸다.
- 하지만, 실험은 현재 사용중인 해결방법은 구성된 해결방법보다 비슷하거나 더 나은 해결방법을 찾을 수 없다고 보여진다. (또는 실현가능한 시간 내에 가능하지 않다.)
- 이 논문에서, 우리는 deep residual learning framework를 소개함으로써 degradation 문제를 해결한다.
- 몇 개의 쌓인 layer 각각이 원하는(desired) 기본(underlying) mapping에 직접 맞기를 바라는 대신에, 이러한 layer가 residual mapping에 명시적으로 맞도록 한다.
- 형식적으로, 원하는 기본 mapping은 $\mathcal{H}(x)$라고 표시하면 쌓인(stacked) 비선형 layer는 $\mathcal{F}(x) := \mathcal{H}(x)-x$의 다른 mapping에 맞도록 한다.
- 원래의 mapping은 $\mathcal{F}(x) + x$로 다시 재구성된다.
- 우리는 원래의 참조되지 않은 mapping을 최적화하는 것보다 residual mapping을 최적화하는 것이 더 쉽다고 가설을 세웠다.
- 극적으로, 만약 identity mapping이 최선이라면, 비선형 layer가 쌓여진 것에 의해 identity mapping을 맞추는 것보다 잔차(residual)을 0으로 밀어내는 것이 더 쉬울 것이다.
- $\mathcal{F}(x) + x$ 식은 "지름길 연결(shortcut connection)"이 있는 feedforward 신경망에 의해 실현될 수 있다. (그림 2)
- <img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FuhPnp%2FbtqvRaMgqx5%2FAzuZ5zkC2I67gzitMNaeYK%2Fimg.png" width=400>

  - 그림 2.
  - residual learning: a building block
- 지름길 연결(shortcut connection)은 1개 이상의 layer를 건너뛴다.
- 우리의 경우, 지름길 연결은 단순히 identity mapping을 수행하고, identity mapping의 결과는 쌓여진 layer의 출력값에 더해진다.
- identity 지름길 연결은 추가적인 파라미터나 계산 복잡성을 추가하지 않는다.
- 전체 네트워크는 SGD(확률적 경사 하강법)에 의해 end-to-end 훈련이 될 수 있다. 그리고 경사 하강법 알고리즘의 종류를 지정하는 매개변수(solver)를 수정하지 않고도 공통 라이브러리를 사용하여 쉽게 구현할 수 있다.
- 우리는 ImageNet의 종합적인 실험을 제시한다. degradation 문제를 보여주고 우리의 방법을 평가하기 위해서.
- 우리는 1), 2)를 보여준다.
  - 1) 매우 깊은 residual net은 최적화하기에 쉽다.
  - 하지만, 단순히 layer를 쌓기만 한 "일반(plain)" 네트워크는 깊이를 증가시키면 훈련 에러가 더 높아진다.
  - 2) deep residual net은 이전 네트워크보다 훨씬 더 나은 결과를 생성함으로써 매우 깊은 깊이로부터 정확도 향상을 쉽게 얻어낸다.
- 유사한 현상이 CIFAR-10 데이터셋에서도 나타나며, 최적화 어려움과 우리 방법의 효과가 특정 데이터셋과 유사하지 않다는 것을 시사한다.
- 100개 이상의 layer가 있는 이 데이터셋에서 성공적으로 훈련된 모델을 제시하고, 1000개 이상의 layer가 있는 모델을 탐색한다.
- ImageNet 분류 데이터셋에서, 우리는 상당히 깊은 residual net에 의해 우수한 결과를 얻었다.
- 우리의 152개 층의 residual net은 ImageNet에 제시된 가장 깊은 네트워크이지만, 여전히 VGG 네트워크보단 복잡성이 낮다.
- 우리의 앙상블(ensemble)은 ImageNet 테스트 데이터셋에서 3.57%의 top-5 오류를 기록했으며, ILSVRC 2015 분류 대회에서 1등을 했다.
- 또한 매우 깊은 표현(representation)은 다른 인식 작업에서 뛰어난 일반화 성능을 갖고 있고, 더 나아가 ILSVRC와 COCO 2015 대회에서 ImageNet detection, COCO detection, COCO segmentation 분야에서 1등을 했다.
- 이러한 강력한 증거는 보여준다. residual learning 원리가 일반적이고, 다른 시각과 비시각 문제에도 적용되는 것을 기대할 수 있다고.

> 요점정리 : 

# 2. Related Work(관련 작업)

# 2.1. Residual Representations(잔여 표현)
- 이미지 인식에서, VLAD는 사전(dictionary)에 대하여 residual vector로 인코딩하는 표현이다. 그리고 Fisher Vector는 VLAD의 확률적인 버전으로 만들 수 있다.
- 둘 다(VLAD, Fisher Vector) 영상 검색(image retrieval) 및 분류(classification)를 위한 강력한 얕은 표현이다.
- 벡터 양자화(vector quantization)에서, residual vector를 인코딩하는 것은 원래의 vector를 인코딩하는 것보다 더 효과적이라고 보여진다.
- 저수준의 vision, computer graphics에서, 편미분 방정식(PDEs, Partial Differential Equations)을 풀기 위해, 널리 사용되는 Multigrid 방법은 시스템을 여러 척도의 하위 문제로 재구성한다. 여기서 각 하위 문제는 결이 거친것(coarser)과 섬세한(finer) 스케일 사이에서 residual 해결방법에 책임이 있다.
- Multigrid 방법의 대안은 계층 기반의 전제 조건 공식(hierarchical basis preconditioning)이다. 이 방법은 두 개의 스케일(척도) 사이에서 residual vector를 나타내는 변수에 의존한다.
- 이러한 solver는 해결방법의 나머지 특성(residual nature)를 잘 인식하지 못하는 standard solver보다는 훨씬 더 빨리 수렴한다.
- 이러한 방법은 좋은 재구성 또는 전제 조건 공식이 최적화를 단순화할 수 있다고 말하고 있다.

# 2.2. Shortcut Connections(지름길 연결)
- 지름길 연결에 대한 훈련과 이론은 오랜시간동안 연구되어왔다.
- 다중 계층 퍼셉트론(MLP, multi-layer perceptron)에 대한 초기 훈련은 네트워크 입력에서 출력으로 연결된 선형 계층을 추가하는 것이다.
- vanishing/exploding gradient 문제를 해결하기 위해서 몇 개의 중간 layer가 보조적인 분류기(auxiliary classifier)에 직접 연결되었다.
- 일부 논문은 layer 응답(response), 기울기(gradient), 전파된 오차(propagated error)를 중심화하는 방법을 제안했다. 그리고 그것들은 지름길 연결에 의해 구현된다.
- "inception"(GoogleNet) layer는 지름길 branch와 몇 개의 더 깊은 branch로 구성된다.
- 우리의 작업과 동시에, "highway networks"는 gating function과 함께 지름길 연결을 제시한다.